# GP2 Pathogenic Variant Analysis

- **Project:** Parkinson’s Disease Pathogenic Variants: Cross-Ancestry Analysis and Microarray Data Validation
- **Version:** Python/3.10.15
    
## Notebook Overview

- Annotating variants across all ancestries utilizing ANNOVAR's ClinVar database for "Pathogenic" variants in established Parkinson's Disease genes (as defined by Blauwendraat et al. 2019)

## CHANGELOG

- 14-NOV-2024: Notebook started

---

# Getting Started

## Importing Packages

In [20]:
import os
import pandas as pd
import numpy as np
import sys as sys

from datetime import date
today = date.today()
date = today.strftime("%d-%b-%Y").upper()

# print out packages and versions
print(f'PACKAGE VERSION ({date})')
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

PACKAGE VERSION (18-NOV-2024)
pandas==2.2.3
numpy==1.25.2


# Run Analysis

- GP2 release 7 (all ancestries)

In [23]:
# define paths/variables

wd = '' # working directory
gp2_tier2 = '' # path to GP2 tier 2 data

plink_exec = '' # path to plink executable
plink2_exec = '' # path to plink2.0 executable

ancestries = ['AAC', 'AFR', 'AJ', 'AMR', 'CAH', 'CAS', 'EAS', 'EUR', 'FIN', 'MDE', 'SAS']

In [ ]:
# utility function

def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command

## convert gp2 release 7 pfiles to bfiles

In [ ]:
# convert the gp2 release 7 pfiles to bfiles for each chromosome of interest for each ancestry group

# list of all ancestries
ancestries = ['AAC', 'AFR', 'AJ', 'AMR', 'CAH', 'CAS', 'EAS', 'EUR', 'FIN', 'MDE', 'SAS']

# list of chromosomes that contain a PD gene of interest
chrom_list = ['1','2','3','4','6','12','14','15','16','17','19','20','21','22']

for anc in ancestries:
    gp2_anc_path = f'{gp2_tier2}/imputed_genotypes/{anc}'
    gp2_anc_samples = f'{gp2_tier2}/raw_genotypes/{anc}/{anc}_release7_vwb.samples'
    print(f'WORKING ON {anc}')
    for chrom in chrom_list:
        plink_cmd = f'{plink2_exec} --pfile {gp2_anc_path}/chr{chrom}_{anc}_release7_vwb \
                      --keep {gp2_anc_samples} \
                      --make-bed --out {workspace}/bfiles/{anc}_{chrom}'
        shell_do(plink_cmd);
    print(f'COMPLETED {anc}')
    print('----------------------------------------')

In [8]:
# create merge files for each ancestry

for anc in ancestries:
    with open(f'{workspace}/bfiles/merge_files_{anc}.txt', 'w') as f:
        for chrom in chrom_list:
            f.write(f'{workspace}/bfiles/{anc}_{chrom}\n')
    f.close()

In [ ]:
# merge chromosomes together for each ancestry

for anc in ancestries:
    plink_merge = f'{plink_exec} --merge-list {workspace}/bfiles/merge_files_{anc}.txt \
                    --make-bed --out {workspace}/bfiles/MERGE_{anc}'

    shell_do(plink_merge);
    print(f'{anc} MERGE COMPLETED')

In [ ]:
# check that all ancestries were merged

!ls {workspace}/bfiles/MERGE*

## annotate the .bim files

In [4]:
%cd {workspace}/bfiles

/home/jupyter/workspace/bfiles


In [10]:
!cat MERGE_AAC.bim MERGE_AFR.bim MERGE_AJ.bim MERGE_AMR.bim MERGE_CAH.bim MERGE_CAS.bim MERGE_EAS.bim MERGE_EUR.bim MERGE_FIN.bim MERGE_MDE.bim MERGE_SAS.bim | awk '{print $2}' | sort | uniq > all_ancestries.txt

In [11]:
%%bash

# input and output file names
input_file="all_ancestries.txt"
output_file="to_annotate.txt"

# create an empty output file
> "$output_file"

# process each line of the input file
while IFS=$':' read -r chr pos ref alt; do
    # write the formatted line to the output file
    echo "$chr $pos $pos $ref $alt" >> "$output_file"
done < "$input_file"

echo "Processing complete. Output saved to $output_file"

Processing complete. Output saved to to_annotate.txt


### download annovar libraries

In [47]:
!/home/jupyter/annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene /home/jupyter/annovar/humandb/
!/home/jupyter/annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp151 /home/jupyter/annovar/humandb/
!/home/jupyter/annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20240917 /home/jupyter/annovar/humandb/
!/home/jupyter/annovar/annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp33a /home/jupyter/annovar/humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the '/home/jupyter/annovar/humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_avsnp151.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_avsnp151.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Fini

### run annovar annotations

In [ ]:
# command to run with just clinvar annotations (no CAD scores)

!/home/jupyter/annovar/table_annovar.pl to_annotate.txt /home/jupyter/annovar/humandb/ -buildver hg38 -out all_ancestries_annotated -protocol refGene,avsnp151,clinvar_20240917 -operation g,f,f -nastring .

NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver hg38 -dbtype refGene -outfile all_ancestries_annotated.refGene -exonsort -nofirstcodondel to_annotate.txt /home/jupyter/annovar/humandb/>
NOTICE: Output files are written to all_ancestries_annotated.refGene.variant_function, all_ancestries_annotated.refGene.exonic_variant_function
NOTICE: Reading gene annotation from /home/jupyter/annovar/humandb/hg38_refGene.txt ... Done with 88819 transcripts (including 21511 without coding sequence annotation) for 28307 unique genes
NOTICE: Processing next batch with 5000000 unique variants in 5000000 input lines
NOTICE: Finished analyzing 1000000 query variants
NOTICE: Finished analyzing 2000000 query variants
NOTICE: Finished analyzing 3000000 query variants
NOTIC

In [ ]:
# command to run CADD score annotations
# - note: this was run after the clinvar annotations were run and manually appended to the ClinVar annotations
#         to run this in the initial run with the ClinVar annotations, include the database in the prior command

# !/home/jupyter/annovar/table_annovar.pl to_annotate.txt /home/jupyter/annovar/humandb/ -buildver hg38 -out all_ancestries_annotated_cadd -protocol refGene,avsnp151,dbnsfp33a -operation g,f,f -nastring .

In [9]:
# check that annotation files were created

!ls -lh all_ancestries*

-rw-r--r-- 1 jupyter jupyter 2.5G Oct  9 23:11 all_ancestries.txt
-rw-r--r-- 1 jupyter jupyter 4.1G Oct 10 17:06 all_ancestries_annotated.hg38_avsnp150_dropped
-rw-r--r-- 1 jupyter jupyter 1.2G Oct 10 17:06 all_ancestries_annotated.hg38_avsnp150_filtered
-rw-r--r-- 1 jupyter jupyter 5.6G Oct 16 06:54 all_ancestries_annotated.hg38_avsnp151_dropped
-rw-r--r-- 1 jupyter jupyter 348M Oct 16 06:54 all_ancestries_annotated.hg38_avsnp151_filtered
-rw-r--r-- 1 jupyter jupyter  85M Oct 10 17:49 all_ancestries_annotated.hg38_clinvar_20221231_dropped
-rw-r--r-- 1 jupyter jupyter 3.5G Oct 10 17:49 all_ancestries_annotated.hg38_clinvar_20221231_filtered
-rw-r--r-- 1 jupyter jupyter 138M Oct 16 07:40 all_ancestries_annotated.hg38_clinvar_20240917_dropped
-rw-r--r-- 1 jupyter jupyter 3.5G Oct 16 07:40 all_ancestries_annotated.hg38_clinvar_20240917_filtered
-rw-r--r-- 1 jupyter jupyter 412M Oct 11 01:13 all_ancestries_annotated.hg38_dbnsfp33a_dropped
-rw-r--r-- 1 jupyter jupyter 3.5G Oct 11 01:13 all_

## filter for pathogenic variants

In [10]:
# genes of interest (PD genes)
genes = ['SNCA', 'PRKN', 'PARK7', 'LRRK2', 'PINK1', 'ATP13A2', 'FBX07', 'GBA', 'PLA2G6', 'VPS35', 'POLG', 'DNAJC6', 'SYNJ1', 'VPS13C', \
         'UCHL1', 'HTRA2', 'GIGYF2', 'EIF4G1', 'DNAJC13', 'TMEM230', 'LRP10']

# create file with these genes to awk from
with open(f'{workspace}/genes_of_interest.txt', 'w') as f:
    for gene in genes:
        f.write(f'{gene}\n')
f.close()

In [12]:
# get only variants that are within genes of interest

!grep -F -f {workspace}/genes_of_interest.txt {workspace}/bfiles/all_ancestries_annotated.hg38_multianno.txt > {workspace}/annotated_selectedgenes.txt

In [24]:
# get column names
annotated_selected_genes_file = open(f'{workspace}/bfiles/all_ancestries_annotated.hg38_multianno.txt', 'r')
col_names = annotated_selected_genes_file.readline().split('\t')
print(col_names)

want_cols = ['Chr','Start','End','Ref','Alt','Gene.refGene','avsnp151','CLNALLELEID','CLNDN','CLNDISDB','CLNREVSTAT','CLNSIG']

['Chr', 'Start', 'End', 'Ref', 'Alt', 'Func.refGene', 'Gene.refGene', 'GeneDetail.refGene', 'ExonicFunc.refGene', 'AAChange.refGene', 'avsnp151', 'CLNALLELEID', 'CLNDN', 'CLNDISDB', 'CLNREVSTAT', 'CLNSIG', 'ONCDN', 'ONCDISDB', 'ONCREVSTAT', 'ONC', 'SCIDN', 'SCIDISDB', 'SCIREVSTAT', 'SCI\n']


In [25]:
annotated_select_genes = pd.read_csv(f'{workspace}/annotated_selectedgenes.txt', sep='\t', header=None, \
                                     names=col_names, usecols=want_cols)
annotated_select_genes

,Chr,Start,End,Ref,Alt,Gene.refGene,avsnp151,CLNALLELEID,CLNDN,CLNDISDB,CLNREVSTAT,CLNSIG
0,chr12,40168848,40168848,A,G,LINC02471;LRRK2,.,.,.,.,.,.
1,chr12,40168854,40168854,C,T,LINC02471;LRRK2,rs541787948,.,.,.,.,.
2,chr12,40168868,40168868,C,A,LINC02471;LRRK2,rs535940699,.,.,.,.,.
3,chr12,40168875,40168875,T,TC,LINC02471;LRRK2,rs576168793,.,.,.,.,.
4,chr12,40168880,40168880,C,A,LINC02471;LRRK2,rs1005738941,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...
464769,chr6,162727742,162727742,C,G,PRKN,rs1362063712,.,.,.,.,.
464770,chr6,162727743,162727743,A,G,PRKN,rs751054800,.,.,.,.,.
464771,chr6,162727745,162727745,C,A,PRKN,rs756731328,.,.,.,.,.
464772,chr6,162727753,162727753,C,T,PRKN,rs556535340,895664,Autosomal_recessive_juvenile_Parkinson_disease_2,"MONDO:MONDO:0010820,MedGen:C1868675,OMIM:60011...","criteria_provided,_single_submitter",Uncertain_significance


In [10]:
# extract only the variants that have a clinical significant == "Pathogenic"

pathogenic_vars = annotated_select_genes[annotated_select_genes['CLNSIG']=='Pathogenic']

# exclude the variants in POLG2 (not a gene of interest)
pathogenic_vars = pathogenic_vars[~(pathogenic_vars['Gene.refGene']=='POLG2')]

# save list of pathogenic variants to an output file
pathogenic_vars.to_csv(f'{workspace}/annotated_pathogenic_vars.txt', sep='\t', header=True, index=False)
pathogenic_vars

(34, 24)


,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,CLNREVSTAT,CLNSIG,ONCDN,ONCDISDB,ONCREVSTAT,ONC,SCIDN,SCIDISDB,SCIREVSTAT,SCI\n
9898,chr12,40310434,40310434,C,T,exonic,LRRK2,.,nonsynonymous SNV,LRRK2:NM_198578:exon31:c.C4321T:p.R1441C,...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
47879,chr15,89319225,89319225,T,A,intronic,POLG,.,.,.,...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
48145,chr15,89321780,89321780,G,A,exonic,POLG,.,nonsynonymous SNV,"POLG:NM_001126131:exon16:c.C2554T:p.R852C,POLG...",...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
48146,chr15,89321792,89321792,C,T,exonic,POLG,.,nonsynonymous SNV,"POLG:NM_001126131:exon16:c.G2542A:p.G848S,POLG...",...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
48530,chr15,89325610,89325610,G,A,exonic,POLG,.,nonsynonymous SNV,"POLG:NM_001126131:exon10:c.C1789T:p.R597W,POLG...",...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
48542,chr15,89325679,89325679,G,A,exonic,POLG,.,nonsynonymous SNV,"POLG:NM_001126131:exon10:c.C1720T:p.R574W,POLG...",...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
48682,chr15,89327166,89327166,C,T,splicing,POLG,NM_001126131:exon7:c.1433+1G>A;NM_002693:exon7...,.,.,...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
48685,chr15,89327201,89327201,C,T,exonic,POLG,.,nonsynonymous SNV,"POLG:NM_001126131:exon7:c.G1399A:p.A467T,POLG:...",...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
48843,chr15,89329041,89329041,G,A,exonic,POLG,.,nonsynonymous SNV,"POLG:NM_001126131:exon4:c.C925T:p.R309C,POLG:N...",...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.
70751,chr1,155235196,155235196,G,A,exonic,GBA,.,nonsynonymous SNV,"GBA:NM_001171811:exon9:c.C1243T:p.R415C,GBA:NM...",...,"criteria_provided,_multiple_submitters,_no_con...",Pathogenic,.,.,.,.,.,.,.,.


In [77]:
# check how many of the pathogenic variants have the term "Parkinson" in ClinVar's 'CLNDN' field

pd_pathogenic_vars = pathogenic_vars[pathogenic_vars['CLNDN'].str.contains("parkinson", case=False, regex=False)]
pd_pathogenic_vars

,Chr,Start,End,Ref,Alt,Gene.refGene,avsnp151,CLNALLELEID,CLNDN,CLNDISDB,CLNREVSTAT,CLNSIG
9898,chr12,40310434,40310434,C,T,LRRK2,rs33939927,16977,not_provided|Autosomal_dominant_Parkinson_dise...,"MedGen:C3661900|MONDO:MONDO:0011764,MedGen:C18...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic
70751,chr1,155235196,155235196,G,A,GBA,rs80356771,19334,"Parkinson_disease,_late-onset|Gaucher_disease-...","MONDO:MONDO:0008199,MedGen:C3160718,OMIM:16860...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic
70948,chr1,155237453,155237453,C,T,GBA,rs78973108,19367,"Parkinson_disease,_late-onset|Gaucher_disease-...","MONDO:MONDO:0008199,MedGen:C3160718,OMIM:16860...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic
71017,chr1,155238215,155238215,T,C,GBA,rs364897,19353,Gaucher_disease-ophthalmoplegia-cardiovascular...,"MONDO:MONDO:0009268,MedGen:C1856476,OMIM:23100...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic
71134,chr1,155239934,155239934,G,A,GBA,rs1141814,19360,"Parkinson_disease,_late-onset|Gaucher_disease-...","MONDO:MONDO:0008199,MedGen:C3160718,OMIM:16860...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic
76909,chr1,20645640,20645640,T,C,PINK1,rs28940285,17447,Autosomal_recessive_early-onset_Parkinson_dise...,"MONDO:MONDO:0011613,MedGen:C1853833,OMIM:60590...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic
77218,chr1,20649109,20649109,C,T,PINK1,rs45539432,17454,not_provided|Autosomal_recessive_early-onset_P...,"MedGen:C3661900|MONDO:MONDO:0011613,MedGen:C18...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic
77228,chr1,20649217,20649217,C,T,PINK1,rs34208370,425331,Autosomal_recessive_early-onset_Parkinson_dise...,"MONDO:MONDO:0011613,MedGen:C1853833,OMIM:60590...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic
113919,chr21,32726907,32726907,C,A,SYNJ1,rs1040540690,1393547,"Developmental_and_epileptic_encephalopathy,_53...","MONDO:MONDO:0033362,MedGen:C4479313,OMIM:61738...","criteria_provided,_single_submitter",Pathogenic
116177,chr22,38132917,38132917,C,A,PLA2G6,rs199935023,39328,Infantile_neuroaxonal_dystrophy|Autosomal_rece...,"MONDO:MONDO:0024457,MedGen:C0270724,OMIM:25660...","criteria_provided,_multiple_submitters,_no_con...",Pathogenic


In [16]:
# check the clinical significance counts for all PD genes of interest

for gene in ['GBA','ATP13A2','PINK1','LRRK2','VPS13C','POLG','VPS35','DNAJC6','PARK7','SYNJ1', \
             'FBXO7','PLA2G6','HTRA2','DNAJC13','UCHL1','SNCA','PRKN','GIGYF2','EIF4G1','TMEM230','LRP10']:
    path_vars_gene = annotated_select_genes[(annotated_select_genes['Gene.refGene'].str.contains(gene))]
    print(gene, path_vars_gene.CLNSIG.value_counts(), '\n')

GBA CLNSIG
.                                                           107049
Uncertain_significance                                          43
Likely_benign                                                   17
Conflicting_classifications_of_pathogenicity                    13
Pathogenic                                                       7
Pathogenic/Likely_pathogenic                                     6
Likely_pathogenic                                                3
Benign                                                           3
Benign/Likely_benign                                             2
Pathogenic/Likely_pathogenic|risk_factor                         1
Conflicting_classifications_of_pathogenicity|risk_factor         1
Conflicting_classifications_of_pathogenicity|other               1
Name: count, dtype: int64 

ATP13A2 CLNSIG
.                                               2805
Likely_benign                                    157
Uncertain_significance              

In [81]:
# save just variant IDs to calculate freqs from

annotated_pathogenic_vars = pd.read_csv(f'{workspace}/annotated_pathogenic_vars.txt', sep='\t')
annotated_pathogenic_vars['snpID'] = annotated_pathogenic_vars['Chr'] + ':' \
                                     + annotated_pathogenic_vars['Start'].astype(str) + ':' \
                                     + annotated_pathogenic_vars['Ref'] + ':' \
                                     + annotated_pathogenic_vars['Alt']
annotated_pathogenic_vars['snpID'].to_csv(f'{workspace}/pathogenic_variant_IDs.txt', header=None, index=None)                                    

In [82]:
# extract pathogenic variants from each ancestry plink files

for anc in ancestries:
    plink_cmd = f'{plink_exec} --bfile {workspace}/bfiles/MERGE_{anc} --extract {workspace}/pathogenic_variant_IDs.txt --make-bed --out {workspace}/bfiles/{anc}_r7_pathogenic_only'
    !{plink_cmd}

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/bfiles/AAC_r7_pathogenic_only.log.
Options in effect:
  --bfile /home/jupyter/workspace/bfiles/MERGE_AAC
  --extract /home/jupyter/workspace/pathogenic_variant_IDs.txt
  --make-bed
  --out /home/jupyter/workspace/bfiles/AAC_r7_pathogenic_only

209368 MB RAM detected; reserving 104684 MB for main workspace.
34460677 variants loaded from .bim file.
1111 people (455 males, 656 females) loaded from .fam.
1086 phenotype values loaded from .fam.
--extract: 2 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1111 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394

In [83]:
# check how many pathogenic variants in each ancestry

for anc in ancestries:
    bim_fn = f'{workspace}/bfiles/{anc}_r7_pathogenic_only.bim'
    if os.path.isfile(bim_fn):
        bim = pd.read_csv(bim_fn, sep='\s+', header=None, names=['CHR','snpID','DIST','POS','ALT','REF'])
        print(anc, bim.shape)
    else:
        print(anc, "no pathogenic vars")

AAC (2, 6)
AFR (8, 6)
AJ (2, 6)
AMR (3, 6)
CAH (2, 6)
CAS (2, 6)
EAS (5, 6)
EUR (19, 6)
FIN (1, 6)
MDE (3, 6)
SAS no pathogenic vars


In [84]:
# run frequencies for case/control per ancestry

for anc in ancestries:
    anc_path = f'{workspace}/bfiles/{anc}_r7_pathogenic_only'
    if os.path.isfile(f'{anc_path}.bim'):
        plink_cmd = f'{plink_exec} --bfile {anc_path} --freq case-control --out {workspace}/freqs/{anc}_path_vars_case_control'
        !{plink_cmd}

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/freqs/AAC_path_vars_case_control.log.
Options in effect:
  --bfile /home/jupyter/workspace/bfiles/AAC_r7_pathogenic_only
  --freq case-control
  --out /home/jupyter/workspace/freqs/AAC_path_vars_case_control

209368 MB RAM detected; reserving 104684 MB for main workspace.
2 variants loaded from .bim file.
1111 people (455 males, 656 females) loaded from .fam.
1086 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1111 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.99955.
--freq case-control: Phen

In [85]:
# run all frequencies per ancestry

for anc in ancestries:
    anc_path = f'{workspace}/bfiles/{anc}_r7_pathogenic_only'
    if os.path.isfile(f'{anc_path}.bim'):
        plink_cmd = f'{plink_exec} --bfile {anc_path} --freq --out {workspace}/freqs/{anc}_path_vars_all'
        !{plink_cmd}

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter/workspace/freqs/AAC_path_vars_all.log.
Options in effect:
  --bfile /home/jupyter/workspace/bfiles/AAC_r7_pathogenic_only
  --freq
  --out /home/jupyter/workspace/freqs/AAC_path_vars_all

209368 MB RAM detected; reserving 104684 MB for main workspace.
2 variants loaded from .bim file.
1111 people (455 males, 656 females) loaded from .fam.
1086 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 1111 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.99955.
--freq: Allele frequencies (founders only) written to
/h

In [90]:
# combine freq calculations

# SAS has no pathogenic vars
all_anc_freqs = pd.DataFrame(columns=['CHR','SNP','A1','A2','MAF','MAF_A','MAF_U'])

for anc in ancestries:
    if os.path.isfile(f'{workspace}/bfiles/{anc}_r7_pathogenic_only.bim'):
        all_freqs = pd.read_csv(f'{workspace}/freqs/{anc}_path_vars_all.frq', sep='\s+', usecols=['CHR','SNP','A1','A2','MAF'])
        case_control_freqs = pd.read_csv(f'{workspace}/freqs/{anc}_path_vars_case_control.frq.cc', sep='\s+', usecols=['CHR','SNP','A1','A2','MAF_A','MAF_U'])
        freqs = pd.merge(all_freqs, case_control_freqs, on=['CHR','SNP','A1','A2'], how='outer')
        all_anc_freqs = all_anc_freqs.merge(freqs, on=['CHR','SNP','A1','A2'], how='outer', suffixes=(None, f'_{anc}'))

all_anc_freqs

,CHR,SNP,A1,A2,MAF,MAF_A,MAF_U,MAF_AAC,MAF_A_AAC,MAF_U_AAC,...,MAF_U_EAS,MAF_EUR,MAF_A_EUR,MAF_U_EUR,MAF_FIN,MAF_A_FIN,MAF_U_FIN,MAF_MDE,MAF_A_MDE,MAF_U_MDE
0,1,chr1:155235196:G:A,A,G,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000348,0.000590,0.000109,NaN,NaN,NaN,NaN,NaN,NaN
1,1,chr1:155236295:G:A,A,G,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,chr1:155237453:C:T,T,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000206,0.000283,0.000054,NaN,NaN,NaN,NaN,NaN,NaN
3,1,chr1:155238206:A:C,C,A,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,chr1:155238215:T:C,C,T,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
5,1,chr1:155238630:G:A,A,G,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000064,0.000094,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
6,1,chr1:155239934:G:A,A,G,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001721,0.003215,0.0
7,1,chr1:16988455:C:T,T,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,chr1:16989961:G:A,A,G,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,chr1:16992345:G:A,A,G,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000026,0.000047,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
# get number of pathogenic variants within each PD gene of interest in each ancestry

gene_path_vars = annotated_pathogenic_vars[['Gene.refGene','avsnp151','snpID']].copy()
gene_path_vars = gene_path_vars.merge(all_anc_freqs, left_on=['snpID'], right_on=['SNP'], how='outer')
gene_path_vars = gene_path_vars.rename(columns={'Gene.refGene':'GENE', 'MAF':'MAF_SAS', 'MAF_A':'MAF_A_SAS', 'MAF_U':'MAF_U_SAS'})

gene_path_vars.fillna('NA').to_csv(f'{workspace}/all_anc_freqs.txt', sep='\t', header=True, index=False)

drop_cols = ['avsnp151','CHR','SNP','A1','A2'] + [f'MAF_A_{anc}' for anc in ancestries] + [f'MAF_U_{anc}' for anc in ancestries]
per_gene_path_vars = gene_path_vars.drop(drop_cols, axis=1)
col_order = ['GENE','snpID'] + [f'MAF_{anc}' for anc in ancestries]
per_gene_path_vars = per_gene_path_vars[col_order]

per_gene_counts = per_gene_path_vars.groupby('GENE').count().reset_index()
per_gene_counts

,GENE,snpID,MAF_AAC,MAF_AFR,MAF_AJ,MAF_AMR,MAF_CAH,MAF_CAS,MAF_EAS,MAF_EUR,MAF_FIN,MAF_MDE,MAF_SAS
0,ATP13A2,5,0,1,0,1,0,0,1,3,0,0,0
1,GBA,7,0,2,0,0,1,0,0,4,0,1,0
2,LRRK2,1,0,0,0,0,0,1,0,1,0,1,0
3,PINK1,3,0,0,0,0,0,0,2,2,1,1,0
4,PLA2G6,3,0,0,0,1,0,1,1,1,0,0,0
5,POLG,8,2,4,1,0,0,0,0,6,0,0,0
6,PRKN,5,0,0,1,1,1,0,1,1,0,0,0
7,SYNJ1,1,0,1,0,0,0,0,0,0,0,0,0
8,TMEM230,1,0,0,0,0,0,0,0,1,0,0,0


In [92]:
# blauwendraat PD gene classifications

gene_classifications = {'SNCA':'very high',\
                        'PRKN':'very high',\
                        'UCHL1':'low',\
                        'PARK7':'very high',\
                        'LRRK2':'very high',\
                        'PINK1':'very high',\
                        'POLG':'high',\
                        'HTRA2':'low',\
                        'ATP13A2':'very high',\
                        'FBXO7':'very high',\
                        'GIGYF2':'low',\
                        'GBA':'very high',\
                        'PLA2G6':'very high',\
                        'EIF4G1':'low',\
                        'VPS35':'very high',\
                        'DNAJC6':'high',\
                        'SYNJ1':'high',\
                        'DNAJC13':'low',\
                        'TMEM230':'low',\
                        'VPS13C':'high',\
                        'LRP10':'low'}

gene_classifications = pd.DataFrame.from_dict(gene_classifications, orient='index').reset_index().rename(columns={'index':'GENE',0:'class'})

In [94]:
# count how many pathogenic variants per gene are present in each ancestry

col_order = ['GENE','class','snpID'] + [f'MAF_{anc}' for anc in ancestries]
per_gene_class_counts = per_gene_counts.merge(gene_classifications, on='GENE', how='outer')[col_order]

class_ord = {'low':2,'high':1,'very high':0}
per_gene_class_counts.sort_values(by='class', key=lambda x: x.map(class_ord)).fillna(0).to_csv(f'{workspace}/per_gene_per_anc_pathogenic_var_counts.txt', sep='\t', index=False)

In [13]:
# compare pathogenic variants with gnomAD annotations
# - annotations were pulled manually for each gene of interest from gnomAD site and then combined to one file

gnomad_info = pd.read_csv(f'{workspace}/gnomAD_selectedgenes_pathogenic_vars.txt', sep='\t')
gnomad_info

,gene,gnomAD_ID,Chromosome,Position,rsIDs,Reference,Alternate,ClinVar_ClnSignificance,ClinVar_varID,allele_count,allele_number,allele_freq
0,ATP13A2,1-16986881-CAG-C,1,16986881,rs1570759415,CAG,C,Pathogenic,660925.0,1,1614048,6.195603e-07
1,ATP13A2,1-16986885-GAGA-G,1,16986885,rs1057519290,GAGA,G,Pathogenic,374887.0,1,1614104,6.195388e-07
2,ATP13A2,1-16986886-A-AG,1,16986886,rs747617559,A,AG,Pathogenic,1968613.0,2,1614040,1.239127e-06
3,ATP13A2,1-16988455-C-T,1,16988455,rs144701072,C,T,Pathogenic,66099.0,15,1613774,9.294982e-06
4,ATP13A2,1-16989739-A-C,1,16989739,rs587777053,A,C,Pathogenic,66098.0,1,1614078,6.195487e-07
...,...,...,...,...,...,...,...,...,...,...,...,...
235,TMEM230,20-5100921-C-A,20,5100921,rs764786986,C,A,Pathogenic,243014.0,5,1613958,3.097974e-06
236,UCHL1,4-41256996-A-C,4,41256996,rs397515634,A,C,Pathogenic,88635.0,2,1614182,1.239018e-06
237,UCHL1,4-41257706-C-CGCT,4,41257706,rs749368841,C,CGCT,Pathogenic,2077519.0,3,1580822,1.897747e-06
238,UCHL1,4-41261751-CAG-C,4,41261751,rs1310363710,CAG,C,Pathogenic,3068443.0,1,1613326,6.198375e-07


In [14]:
annotated_path_vars = pd.read_csv(f'{workspace}/annotated_pathogenic_vars.txt', sep='\t')

gnomad_info['in_gp2'] = gnomad_info['rsIDs'].isin(annotated_path_vars['avsnp151'].tolist())
gnomad_info.to_csv(f'{workspace}/gnomAD_inGP2.txt', sep='\t', header=True, index=False)